# Pracownia z analizy numerycznej, zadanie P1.16

# Metoda Newtona

In [ ]:
using PlotlyJS
include("program.jl")

## Małe układy równań
Zaprezentowane zostaną przebiegi metody Newtona dla przykładowych układów równań. Na końcu sekcji znajdują się wykresy porównujące zmianę błędu względnego szukanego rozwiązania oraz zmianę wartości funkcji dla wszystkich funkcji podanych poniżej

### Przykład 1
\begin{equation*}
F_1(x) = 
\left(\begin{matrix}
	(x_1)^2 + (x_2)^2 - 25 \\
    (x_1)^2 - x_2 - 2
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(1, true)
xtrace1 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₁")
ftrace1 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₁")

### Przykład 2
\begin{equation*}
F_2(x) = 
\left(\begin{matrix}
	x_1 + x_2 + x_3 \\
	(x_1)^2 + (x_2)^2 + (x_3)^2 - 2 \\
    x_1(x_2 + x_3) + 1
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(2, true)
xtrace2 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₂")
ftrace2 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₂")

### Przykład 3
\begin{equation*}
F_3(x) = 
\left(\begin{matrix}
	x_1 x_2 - (x_3)^2 - 1 \\
	x_1 x_2 x_3 + (x_2)^2 - (x_1)^2 - 2 \\
    e^{x_1} + x_3 - e^{x_2} - 3
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(3, true)
xtrace3 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₃")
ftrace3 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₃")

### Przykład 4
\begin{equation*}
F_4(x) = 
\left(\begin{matrix}
	(x_1)^2 \\
	(x_2)^2 
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(4, true)
xtrace4 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₄")
ftrace4 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₄")

### Przykład 5
\begin{equation*}
F_5(x) = 
\left(\begin{matrix}
	(x_1)^3 + 7(x_2)^2 - 7 \\
	(x_1)^2 + (x_2)^2 - 1
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(5, true)
xtrace5 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₅")
ftrace5 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₅")

### Przykład 6
\begin{equation*}
F_6(x) = 
\left(\begin{matrix}
    (x_1)^2 + \sin x_1 \cos x_2 \\
    (x_2)^2 + \cos x_1 \sin x_2
\end{matrix}\right)
\end{equation*}

In [ ]:
iters, exs, efs = run_set(6, true)
xtrace6 = scatter(; x=1:iters, y=-log10(exs), mode="lines+markers", name="F₆")
ftrace6 = scatter(; x=1:iters, y=-log10(efs), mode="lines+markers", name="F₆")

In [ ]:
layout = Layout(;title="Dokładność względna przybliżenia", yaxis_title="Wskaźnik dokładności", xaxis_title="Iteracja")
plot([xtrace1, xtrace2, xtrace3, xtrace4, xtrace5, xtrace6], layout)

In [ ]:
layout = Layout(;title="Norma wartości funkcji", yaxis_title="Wskaźnik dokładności", xaxis_title="Iteracja")
plot([ftrace1, ftrace2, ftrace3, ftrace4, ftrace5, ftrace6], layout)

##  Duże układy równań
Tutaj porównam wpływ wielkości układu równań oraz przybliżenia początkowego na zbieżność metody Newtona oraz ilość iteracji potrzebną do znalezienia pierwiastka. Dla każdego z czterech prototypów funkcji wykonuję obliczenia dla wielkości układu $ n $ od $ 2 $ do $ 20 $ i porównuję zbieżność oraz ilość iteracji dla przybliżeń początkowych
$$
x^{(0)} =
\left(\begin{matrix}
    x_1 \\
    x_2 \\
    \vdots \\
    x_n
\end{matrix}\right), \quad \text{ dla } x_i \in (1-\delta, 1+\delta)
$$
Dla parametru $ \delta = 0.1, 0.2, 0.5, 1.0 $. Każda z funkcji rozważanych poniżej ma rozwiązanie 
$$ \bar{x} =  
\left(\begin{matrix}
    1 \\
    \vdots \\
    1
\end{matrix}\right)
$$

#### Uwaga
Każdy z poniższych testów może liczyć się nawet kilka minut.

### Układ 1
\begin{align*}
F(x) &= 
	\left(\begin{matrix}
		f_1(x) - f_1(\bar{x}) \\ 
		f_2(x) - f_2(\bar{x}) \\ 
		\vdots\\ 
		f_n(x) - f_n(\bar{x})
	\end{matrix}\right), && \text{dla} \quad f_i(X) = a_{ik}X_iX_k, \quad a_{ik} \in (-10, 10)
\end{align*}

In [ ]:
deltas = [0.1, 0.2, 0.5, 1.0]
itraceF = Array{GenericTrace}(size(deltas, 1))
dtraceF = Array{GenericTrace}(size(deltas, 1))
for i in eachindex(deltas)
    @printf "Obliczam dla δ = %.3f\n" deltas[i]
    iters, divgs = convergence_analysis("polynomial", deltas[i], verbose = false)
    itraceF[i] = scatter(; x=2:20, y=iters, mode="lines+markers", name="δ = $(deltas[i])")
    dtraceF[i] = scatter(; x=2:20, y=divgs.*100, mode="lines+markers", name="δ = $(deltas[i])")
end

In [ ]:
ilayout = Layout(;title="Iteracja", yaxis_title="Iteracje", xaxis_title="Wielkość układu równań: n")
plot(itraceF, ilayout)

In [ ]:
dlayout = Layout(;title="Rozbieżność", yaxis_title="Przebiegi rozbieżne [%]", xaxis_title="Wielkość układu równań: n")
plot(dtraceF, dlayout)

### Układ 2
\begin{align*}
G(x) &=
	\left(\begin{matrix}
	g_1(x) - g_1(\bar{x}) \\ 
	g_2(x) - g_2(\bar{x}) \\ 
	\vdots\\ 
	g_n(x) - g_n(\bar{x})
\end{matrix}\right), && \text{dla} \quad g_i(X) = (x_i)^2\ + \sum_{k=1, k\neq i}^{n}\sin{x_i}cos{x_k}
\end{align*}

In [ ]:
itraceG = Array{GenericTrace}(size(deltas, 1))
dtraceG = Array{GenericTrace}(size(deltas, 1))
for i in eachindex(deltas)
    @printf "Obliczam dla δ = %.3f\n" deltas[i]
    iters, divgs = convergence_analysis("sin_cos", deltas[i], verbose = false)
    itraceG[i] = scatter(; x=2:20, y=iters, mode="lines+markers", name="δ = $(deltas[i])")
    dtraceG[i] = scatter(; x=2:20, y=divgs.*100, mode="lines+markers", name="δ = $(deltas[i])")
end

In [ ]:
plot(itraceG, ilayout)

In [ ]:
plot(dtraceG, dlayout)

### Układ 3
\begin{align*}
H(x) &= 
	\left(\begin{matrix}
	h_1(x) - h_1(\bar{x}) \\ 
	h_2(x) - h_2(\bar{x}) \\ 
	\vdots\\ 
	h_n(x) - h_n(\bar{x})
\end{matrix}\right), && \text{dla} \quad h_i(X) = 
		\begin{cases}  
			\sin{x_i}\sin{x_{i+1}}\cos{x_{i+2}}, & \text{ gdy } i \leq n-2 \\
			\sin{x_{n-1}}\sin{x_{n}}\cos{x_{1}}, & \text{ gdy } i = n - 1 \\
			\sin{x_n}\sin{x_{1}}\cos{x_{2}}, & \text{ gdy } i = n
		\end{cases}
\end{align*}

In [ ]:
itraceH = Array{GenericTrace}(size(deltas, 1))
dtraceH = Array{GenericTrace}(size(deltas, 1))
for i in eachindex(deltas)
    @printf "Obliczam dla δ = %.3f\n" deltas[i]
    iters, divgs = convergence_analysis("sin_cos_2", deltas[i], verbose = false)
    itraceH[i] = scatter(; x=2:20, y=iters, mode="lines+markers", name="δ = $(deltas[i])")
    dtraceH[i] = scatter(; x=2:20, y=divgs.*100, mode="lines+markers", name="δ = $(deltas[i])")
end

In [ ]:
plot(itraceH, ilayout)

In [ ]:
plot(dtraceH, dlayout)

### Układ 4
\begin{align*}
V(x) &=
	\left(\begin{matrix}
		v_1(x) - v_1(\bar{x}) \\ 
		v_2(x) - v_2(\bar{x}) \\ 
		\vdots\\ 
		v_n(x) - v_n(\bar{x})
	\end{matrix}\right), && \text{dla} \quad v_i(x) = x_1 + (i+1)x_2 + \ldots + (i+1)^{n-1}x_n
\end{align*}

In [ ]:
itraceV = Array{GenericTrace}(size(deltas, 1))
dtraceV = Array{GenericTrace}(size(deltas, 1))
for i in eachindex(deltas)
    @printf "Obliczam dla δ = %.3f\n" deltas[i]
    iters, divgs = convergence_analysis("vandermonde", deltas[i], verbose = false)
    itraceV[i] = scatter(; x=2:20, y=iters, mode="lines+markers", name="δ = $(deltas[i])")
    dtraceV[i] = scatter(; x=2:20, y=divgs.*100, mode="lines+markers", name="δ = $(deltas[i])")
end

In [ ]:
plot(itraceV, ilayout)

In [ ]:
plot(dtraceV, dlayout)

# Metoda eliminacji Gaussa

In [ ]:
compare_gauss()